In [1]:
from ipywwt import WWTWidget

In [2]:
import ipywidgets

output = ipywidgets.Output()

# @output.capture()
# def on_msg(_widget, msg, buffers):
#     print(f'Received: {type(msg)}')

@output.capture()
def wwt_cb(wwt, updated):
    print(updated)

wwt1 = WWTWidget()

# wwt1.on_msg(on_msg)
wwt1.set_selection_change_callback(wwt_cb)

# wwt1.on_msg(lambda w, m, b: wwt1._on_app_message_received(m, b))
    

ipywidgets.VBox([wwt1, output])

In [3]:
from astropy.table import Table
OEC = '/Users/nmearl/Downloads/RW_cluster_oscillation_0_updated_radec.csv'
table = Table.read(OEC, delimiter=',', format='ascii.basic')

wwt1.layers.add_table_layer(table=table, frame='Sky',
                           lon_att='ra', lat_att='dec')

<TableLayer with 89 markers>

In [4]:
wwt1.layers[0].size_scale = 100
wwt1.layers[0].type = "gaussian"
wwt1.layers[0].color = "#00FF00"

In [6]:
wwt1.get_fov()

<Quantity 28.69781401 deg>

Layout()

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u
# wwt1.center_on_coordinates(SkyCoord.from_name('Andromeda'), instant=True)
wwt1.center_on_coordinates(SkyCoord(285.0130833333333 * u.deg,40.22075 * u.deg), instant=True)
# wwt1.center_on_coordinates(SkyCoord(180 * u.deg, 25 * u.deg, frame='icrs'))

In [ ]:
# wwt1.background = "WMAP ILC 5-Year Cosmic Microwave Background"
wwt1.background = "SDSS: Sloan Digital Sky Survey (Optical)"
wwt1.foreground = "SDSS: Sloan Digital Sky Survey (Optical)"

In [ ]:
wwt2 = WWTWidget()
wwt2

In [ ]:
wwt1.layers[0].id